In [62]:
# Imports dos módulos + caminho da pasta dos arquivos
import os
import sys
import pandas as pd
sys.path.append('../src')  # 
from src.extract_data import get_data_from_api
from src.load_data import get_data_from_files
from config.config import API_URL, CSV_FILE_PATH1, XLS_FILE_PATH2

In [103]:
# chamando a funçao que extrai os dados da api e guarda num df
api_url = API_URL
df_api = get_data_from_api(api_url)
# Criando uma copia dos dados da api
df_api_copy = df_api.copy()

# chamando a função que lê os arquivos csv e xls
df_csv, df_xls = get_data_from_files()
# Criando uma copia dos arquivos csv e xls
df_csv_copy = df_csv.copy()

df_xls_copy = df_xls.copy()

status: 200


In [104]:
# Tratando os dados xls
# definindo o numero de linhas para dropar
n = 2
# Pegando a partir da 4 linha e fazedno o reset do index
df_xls_copy = df_xls_copy.iloc[n:].reset_index(drop=True)
# Definindo o nome da primeira linha como header
df_xls_copy.columns = df_xls_copy.iloc[0]
# Removendo a linha que era header e fazendo o reset do index
df_xls_copy = df_xls_copy[1:].reset_index(drop=True)
# selcionando apenas as duas primeiras colunas do df
df_xls_copy = df_xls_copy.iloc[:, :2]

In [105]:
df_xls_copy['Data'] = pd.to_datetime(df_xls_copy['Data'], format='%m/%Y', errors='coerce')

date_range = pd.date_range(start=df_xls_copy['Data'].min(), end=df_xls_copy['Data'].max(), freq='MS')
date_df = pd.DataFrame(date_range, columns=['Data'])
merged_df = date_df.merge(df_xls_copy, on='Data', how='left')
merged_df['Data'] = merged_df['Data'].dt.strftime("%m/%Y")

In [ ]:
# Mudando o tipo de dados para float64
merged_df['À vista R$'] = merged_df['À vista R$'].astype('float64')

# Complete o campo de valor que não estejam preenchidos com o valor do mês anterior;
merged_df.set_index('Data',inplace=True)
merged_df['À vista R$'] = merged_df['À vista R$'].ffill().infer_objects(copy=False)
merged_df['À vista R$'] = merged_df['À vista R$'].round(2)

In [108]:
merged_df.shape

(21, 1)